<a href="https://colab.research.google.com/github/Shraddha-Ingale34/Shraddha-Ingale34.github.io/blob/main/Assignment3_Shraddha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [665]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from collections import Counter
import os
import glob as glob
import string
import re
import random
from bs4 import BeautifulSoup
from sklearn.model_selection import KFold

In [666]:
# Upload the file on drive and then copy that path
imdb_labelFile = open(r"/content/drive/MyDrive/Naive Bayes/imdb_labelled.txt", "r")
def getReviewTargetFromFile(file):
    lines=file.readlines()
    file.close()
    reviewsentimentList=[]
    for l in lines:
        x=l.split('\t')
        reviewsentimentList.append([str.lstrip(str.rstrip(x[0])),str.lstrip(str.rstrip(x[1]))])
    return reviewsentimentList
 
rtList=getReviewTargetFromFile(imdb_labelFile)
rtDF=pd.DataFrame(rtList,columns=['REVIEW','TARGET'])
rtDF.to_csv('imdb.csv', index = None)
imdb_data = pd.read_csv('imdb.csv')
imdb_data.head()

,REVIEW,TARGET
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [667]:
imdb_data.count()

REVIEW    1000
TARGET    1000
dtype: int64

In [668]:
#sentiment count
imdb_data['TARGET'].value_counts()

1    500
0    500
Name: TARGET, dtype: int64

In [669]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [670]:
df_train = rtDF.sample(frac=0.6)
df_dev = rtDF.sample(frac=0.2)
df_test = rtDF.sample(frac=0.2)

In [671]:
print("Size of Train Data :", len(df_train))
print("Size of Development Data :", len(df_dev))
print("Size of Test Data :", len(df_test))

Size of Train Data : 600
Size of Development Data : 200
Size of Test Data : 200


In [672]:
def Word_Count(data):
    dev_review = data
    unique_words_dev = []
    for review in dev_review:
        for word in review.split(' '):
                unique_words_dev.append(word)
    unique_words_dev = Counter(unique_words_dev)
    unique_words_dev = Counter({k: n for k, n in unique_words_dev.items() if n > 5}) #removing rare words that occure more than 5
    return unique_words_dev

In [673]:
def probability_wordOccur(data,word,nOfdocs):
    idct_data=dict(data)
    if word in idct_data.keys():
        return idct_data[word] / nOfdocs
    else: return 0

In [674]:
wrdcnt = Word_Count(list(list(zip(*rtDF.values))[0]))

In [675]:
print("The P['the'] i.e the probablity of 'the' given no of docs is: ",probability_wordOccur(wrdcnt,'the',len(imdb_data)))

The P['the'] i.e the probablity of 'the' given no of docs is:  0.655


In [676]:
def conditional_probability(X,y,word,givenFlag):
    group = np.unique(y)
    separated = dict()
    for sentiment in group:
        separated[sentiment] = X[np.where(y == sentiment)]
    countpos = dict(Word_Count(separated['1']))
    countneg = dict(Word_Count(separated['0']))
    
    if givenFlag == '1':
        if word in countpos.keys():
            return countpos[word]/ len(separated['1'])
        else: return 0
    else:
        if word in countneg.keys():
            return countneg[word]/ len(separated['0'])
        else: return 0

In [677]:
X_review = df_train['REVIEW'].values
y_target = df_train['TARGET'].values

print(" The P['the'|positive] is:", conditional_probability(X_review,y_target,'the','1'))

 The P['the'|positive] is: 0.7123745819397993


In [678]:
class NaiveBayes:
    def __init__(self,X,y,test,alpha,flag):
        self.alpha = alpha
        self.X = X
        self.y = y
        self.test = test
        self.target = np.unique(y)
        self.flag = flag
        
    
    def conditional_prob(self,word,target):
        if target == '1':
            if word in self.countpositive.keys():
                return self.countpositive[word]/ len(self.separated['1'])
            else: return 0
        else:
            if word in self.countnegative.keys():
                return self.countnegative[word]/ len(self.separated['0'])
            else: return 0
            
    def probability_wordOcc(self,word):
        if word in self.wordcnt.keys():
            return self.wordcnt[word] / len(self.X)
        else: return 0
        
    def bayes(self,word,target):
        p_word_gvnSenti = self.conditional_prob(word,target)
        
        p_target = list(self.y).count(target) / len(list(self.y))
        
        p_word = self.probability_wordOcc(word)
        
        if p_word == 0:
            p_word = 0.0000001
        if self.alpha == 0:
            return (p_word_gvnSenti*p_target)/p_word  #without smoothing
        else:
            if word in self.wordcnt.keys():
                count = self.wordcnt[word]
            else: count = 1
            return ((p_word_gvnSenti*p_target) + self.alpha) / (p_word + count) #with smoothing
        
        
    def NB_predict(self):
        result = 1
        y_pred = []
        positive = 0
        negative = 0
        self.separated = dict()
        positivedi = dict()
        negativedi = dict()
        
        self.wordcnt = dict(Word_Count(self.X)) #vocab list
        
        for i in self.target:
            self.separated[i] = self.X[np.where(self.y == i)] 
        self.countpositive = dict(Word_Count(self.separated['1']))
        self.countnegative = dict(Word_Count(self.separated['0']))
        
        if self.flag != True:
            for review in self.test:
                for senti in self.target:
                    for word in review.split():
                        prob = self.bayes(word,senti)
                        result = result * prob
                    if senti == 1:
                        positive = result
                    else:
                        negative = result

                if positive > negative:
                    y_pred.append('1')
                else: y_pred.append('0')

            return y_pred
        else:
            for word in self.countpositive.keys(): 
                prob = self.bayes(word,1)
                positivedi[word] = prob
            for word in self.countnegative.keys(): 
                prob = self.bayes(word,0)
                negativedi[word] = prob
            return positivedi,negativedi

In [679]:
X_dev = df_dev['REVIEW'].values
y_dev = df_dev['TARGET'].values

In [680]:
NBC = NaiveBayes(X_review,y_target,X_dev,0,0)
ypred = NBC.NB_predict()

In [681]:
ypred = np.array(ypred)

In [682]:
def calculate_accuracy(actual,predicted):
    no_correctOfPredict = 0.0
    for y_true, y_pred in zip(actual,predicted):
        if y_true == y_pred:
            no_correctOfPredict += 1
    return no_correctOfPredict / float(len(actual)) * 100.0

In [683]:
print("Accuracy on the development Dataset without smoothing is:",calculate_accuracy(y_dev,ypred))

Accuracy on the development Dataset without smoothing is: 42.5


In [684]:
df_train_short = df_train.sample(frac=0.8)
len(df_train_short)

480

In [685]:
def cross_validation(data,k,alpha):
    dev = 1/k
    score = []
    for i in range(1, k + 1):
        df_dev_cv = data.sample(frac=dev, random_state=i, replace=False).copy(deep=True)
        df_train_cv = data.drop(df_dev_cv.index, axis=0).copy(deep=True)
        X_dev = df_dev_cv['REVIEW'].values
        y_dev = df_dev_cv['TARGET'].values
        X_train = df_train_cv['REVIEW'].values
        y_train = df_train_cv['TARGET'].values
        NBC = NaiveBayes(X_train,y_train,X_dev,alpha,flag= False)
        ypred = np.array(NBC.NB_predict())
        score.append(calculate_accuracy(y_dev,ypred))
    return score

In [686]:
score = cross_validation(df_train_short,5,1)

In [687]:
print("Accuracy on development dataset with smoothing with alpha 1 : {} %".format(np.mean(score)))

Accuracy on development dataset with smoothing with alpha 1 : 50.416666666666664 %


In [688]:
score12 = cross_validation(df_train_short,5,0.001)

In [689]:
print("Accuracy on development dataset with smoothing with alpha 0.001 : {} %".format(np.mean(score12)))

Accuracy on development dataset with smoothing with alpha 0.001 : 50.416666666666664 %


In [690]:
positive=dict()
wordcount = dict(wrdcnt)
NBC = NaiveBayes(X_train,y_train,0,1,flag=True)
positive, negative = NBC.NB_predict()

In [691]:
positive

{'10': 0.09983361064891846,
 'A': 0.07679508511455267,
 'All': 0.1663893510815308,
 'But': 0.07130972189208462,
 'I': 0.006014072930657739,
 'If': 0.0831946755407654,
 'It': 0.04340591767344281,
 "It's": 0.07130972189208462,
 'The': 0.008460475478721904,
 'This': 0.021241193755089035,
 'a': 0.004321801326793008,
 'about': 0.038397542557276336,
 'acting': 0.05872565332289322,
 'actors': 0.12479201331114809,
 'all': 0.04753981459472308,
 'also': 0.07130972189208462,
 'an': 0.04340591767344281,
 'and': 0.0038997504159733777,
 'are': 0.019198771278638168,
 'as': 0.016638935108153077,
 'at': 0.03119800332778702,
 'be': 0.03220439053190918,
 'because': 0.06655574043261231,
 'best': 0.07130972189208462,
 'better': 0.14261944378416924,
 'both': 0.12479201331114809,
 'but': 0.026272002802346966,
 'by': 0.03565486094604231,
 'can': 0.06655574043261231,
 'characters': 0.06239600665557404,
 'do': 0.09983361064891846,
 'enjoyed': 0.14261944378416924,
 'ever': 0.04991680532445923,
 'every': 0.110926

In [692]:
negative

{'-': 0.05872565332289322,
 'A': 0.07679508511455267,
 'But': 0.07130972189208462,
 'I': 0.006014072930657739,
 "I've": 0.11092623405435385,
 'It': 0.04340591767344281,
 "It's": 0.07130972189208462,
 'The': 0.008460475478721904,
 'There': 0.09075782786265316,
 'This': 0.021241193755089035,
 'a': 0.004321801326793008,
 'about': 0.038397542557276336,
 'acting': 0.05872565332289322,
 'all': 0.04753981459472308,
 'an': 0.04340591767344281,
 'and': 0.0038997504159733777,
 'any': 0.07130972189208462,
 'anyone': 0.09983361064891846,
 'are': 0.019198771278638168,
 'as': 0.016638935108153077,
 'at': 0.03119800332778702,
 'bad': 0.0415973377703827,
 'bad.': 0.09983361064891846,
 'be': 0.03220439053190918,
 'because': 0.06655574043261231,
 'but': 0.026272002802346966,
 'by': 0.03565486094604231,
 'can': 0.06655574043261231,
 'could': 0.07130972189208462,
 "didn't": 0.07679508511455267,
 "don't": 0.11092623405435385,
 'even': 0.06655574043261231,
 'ever': 0.04991680532445923,
 'film': 0.0155990016

In [693]:
df_pos = pd.DataFrame.from_dict(positive, orient='index')
df_neg = pd.DataFrame.from_dict(negative, orient='index')

In [694]:
df_pos.sort_values(by=0,ascending=False).head(10)

,0
funny,0.166389
give,0.166389
All,0.166389
better,0.142619
liked,0.142619
enjoyed,0.142619
wonderful,0.124792
both,0.124792
way,0.124792
actors,0.124792


In [695]:
df_neg.sort_values(by=0,ascending=False).head(10)

,0
worst,0.166389
waste,0.166389
those,0.142619
know,0.142619
little,0.124792
script,0.124792
them,0.124792
totally,0.124792
work,0.110926
don't,0.110926


In [696]:
score = cross_validation(df_test,5,1)

In [697]:
print("Accuracy on test data with smoothing with alpha 1 : {} %".format(np.mean(score)))

Accuracy on test data with smoothing with alpha 1 : 58.0 %
